# 직전 2개월 / 직전 3개월 테이블 생성
<hr>

## 1. 데이터 불러오기

In [1]:
import pandas as pd
master = pd.read_csv('./csv/Master.csv')
product = pd.read_csv('./csv/Pruduct.csv')
session = pd.read_csv('./csv/Session.csv')

C:\Users\gmlwl\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### - 상품구매 테이블에서 브랜드명 전처리

In [2]:
new = []
for x in product.PD_BRA_NM:
    new.append(x.strip('[]'))
    
product['PD_BRA_NM'] = new
product.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,4139680,7605037,12,642112,색상:워터멜론,바비브라운,"39,000",1
1,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,"39,000",2
2,4142395,6158159,85,179538,(not set),아베다,"39,000",1
3,4144914,7935714,12,554336,색상:블랙|사이즈:160cm(12~13세) / 1개,아디다스 키즈,"39,000",1
4,4144917,6406509,78,190306,5개,데코르테,"39,000",5


<br>

## 2. Master / Product / Session 테이블 Merge

In [3]:
master_product = master.merge(product, on='PD_C')

In [4]:
mps = master_product.merge(session, on=['CLNT_ID', 'SESS_ID'], how = 'outer')

In [5]:
mps.head()

,PD_C,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_ID,SESS_ID,HITS_SEQ,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,1489284,2719478,269,색상:블랙|사이즈:SM[90] / 1개,언더아머(스포츠),"49,000",1,577,20180815,306.0,"2,099",mobile,Seoul,Seoul
1,301738,아디다스 D2M K 숏 (CV3341) 여성 숏반바지/숏팬츠 - 100,스포츠패션,여성일반스포츠의류,여성일반스포츠바지,1489284,2719478,309,사이즈:085 / 1개,아디다스(의류),"20,900",1,577,20180815,306.0,"2,099",mobile,Seoul,Seoul
2,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,4671890,2659813,230,색상:블랙|사이즈:MD[95] / 1개,언더아머(스포츠),"49,000",1,590,20180816,262.0,"2,544",mobile,Chungcheongnam-do,Asan-si
3,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,559585,2330751,19,색상:블랙|사이즈:MD[95] / 1개,언더아머(스포츠),"49,000",1,388,20180822,21.0,284,mobile,Gyeonggi-do,Ansan-si
4,254261,여성 로퍼 AOS804 2종 택 1 (5cm) - 02.AOS804PA30_WHIT...,패션잡화,여성화,여성로퍼,559585,2330751,19,색상:01.AOS804PA10_BLACK|사이즈:240 / 1개,소다,"63,000",1,388,20180822,21.0,284,mobile,Gyeonggi-do,Ansan-si


#### - PD_BUY_CT에서 str형으로 되어있는 레코드 numpy.int64로 변형

In [6]:
type(mps['PD_BUY_CT'][3844054])

str

In [7]:
list1 = [3844054, 4197166]
for i in list1:
    mps['PD_BUY_CT'][i] = mps['PD_BUY_CT'][i].replace(',','')
    mps['PD_BUY_CT'][i] = int(mps['PD_BUY_CT'][i])

C:\Users\gmlwl\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\gmlwl\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [8]:
import numpy as np
mps['PD_BUY_CT'] = np.dtype('int64').type(mps['PD_BUY_CT'])

<hr>

# 3. 직전 2개월 데이터 기술통계 테이블 생성 (1번 테이블)
## 3-1) 4월 ~ 5월 상품군별(상품소분류 기준) 판매량 기술통계 테이블


In [9]:
# 날짜 비교를 위해 SESS_DT를 string 형으로 변환
mps['SESS_DT'] = mps['SESS_DT'].astype(str)
mps

,PD_C,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_ID,SESS_ID,HITS_SEQ,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,1489284,2719478,269,색상:블랙|사이즈:SM[90] / 1개,언더아머(스포츠),"49,000",1,577,20180815,306.0,"2,099",mobile,Seoul,Seoul
1,301738,아디다스 D2M K 숏 (CV3341) 여성 숏반바지/숏팬츠 - 100,스포츠패션,여성일반스포츠의류,여성일반스포츠바지,1489284,2719478,309,사이즈:085 / 1개,아디다스(의류),"20,900",1,577,20180815,306.0,"2,099",mobile,Seoul,Seoul
2,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,4671890,2659813,230,색상:블랙|사이즈:MD[95] / 1개,언더아머(스포츠),"49,000",1,590,20180816,262.0,"2,544",mobile,Chungcheongnam-do,Asan-si
3,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,559585,2330751,19,색상:블랙|사이즈:MD[95] / 1개,언더아머(스포츠),"49,000",1,388,20180822,21.0,284,mobile,Gyeonggi-do,Ansan-si
4,254261,여성 로퍼 AOS804 2종 택 1 (5cm) - 02.AOS804PA30_WHIT...,패션잡화,여성화,여성로퍼,559585,2330751,19,색상:01.AOS804PA10_BLACK|사이즈:240 / 1개,소다,"63,000",1,388,20180822,21.0,284,mobile,Gyeonggi-do,Ansan-si
5,206280,여성 스판배색와이드핏8부밴딩바지 (MQPT3213) - 그레이 / 66,남성의류,남성의류하의,남성캐주얼바지,559585,2330751,19,색상:그레이|사이즈:66 / 1개,마인드브릿지,"39,000",1,388,20180822,21.0,284,mobile,Gyeonggi-do,Ansan-si
6,171995,남성 레이어링 티셔츠(MRTS3160) - 네이비 / 105,남성의류,남성의류상의,남성티셔츠,559585,2330751,19,색상:네이비|사이즈:105 / 1개,마인드브릿지,"19,000",1,388,20180822,21.0,284,mobile,Gyeonggi-do,Ansan-si
7,389463,여성 스판 자수장식 루즈핏셔츠(MRWS320A) - 라이트블루 / FREE,남성의류,남성의류상의,남성티셔츠,559585,2330751,19,색상:오프화이트|사이즈:FREE / 1개,마인드브릿지,"29,000",1,388,20180822,21.0,284,mobile,Gyeonggi-do,Ansan-si
8,62282,여자 플라워덧신 2족선물세트 17403-2set,속옷/양말/홈웨어,여성양말류,여성일반양말,2820547,733100,23,4개,안스엉클(양말),"10,000",4,36,20180918,38.0,"3,800",mobile,Gyeongsangnam-do,Sacheon-si
9,61729,88A2933253배트맨스웨트티 - 블랙 / 130,유아동의류,유아의류상의,영유아티셔츠/탑,221797,68725,210,색상:블랙|사이즈:140 / 1개,래핑차일드,"35,000",1,61,20180929,190.0,"3,710",mobile,Gyeonggi-do,Goyang-si


In [10]:
m0405_SUM = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180531"').groupby('CLAC3_NM').sum()[['PD_BUY_CT']]
m0405_MEAN = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180531"').groupby('CLAC3_NM').mean()[['PD_BUY_CT']]
m0405_VAR = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180531"').groupby('CLAC3_NM').var()[['PD_BUY_CT']]
m0405_MEDIAN = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180531"').groupby('CLAC3_NM').median()[['PD_BUY_CT']]
m0405_STD = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180531"').groupby('CLAC3_NM').std()[['PD_BUY_CT']]
m0405_MAX = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180531"').groupby('CLAC3_NM').max()[['PD_BUY_CT']]
m0405_MIN = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180531"').groupby('CLAC3_NM').min()[['PD_BUY_CT']]

In [11]:
m0405 = pd.concat([m0405_SUM, np.round(m0405_MEAN,2), np.round(m0405_VAR,2), m0405_MEDIAN, np.round(m0405_STD,2), m0405_MAX, m0405_MIN], axis=1)
m0405.columns=['m0405_SUM', 'm0405_MEAN', 'm0405_VAR', 'm0405_MEDIAN', 'm0405_STD', 'm0405_MAX', 'm0405_MIN']
m0405

,m0405_SUM,m0405_MEAN,m0405_VAR,m0405_MEDIAN,m0405_STD,m0405_MAX,m0405_MIN
CLAC3_NM,,,,,,,
2단우산,110,1.41,1.47,1.0,1.21,7,1
3단우산,806,1.20,1.02,1.0,1.01,11,1
BB/파운데이션/컴팩트류,24690,1.09,0.22,1.0,0.47,20,1
DIY가구,467,13.34,554.88,1.0,23.56,92,1
DSLR,227,1.00,0.00,1.0,0.00,1,1
LCD,2,1.00,0.00,1.0,0.00,1,1
LED,536,1.06,0.39,1.0,0.63,9,1
OLED,2,1.00,0.00,1.0,0.00,1,1
PC부품,1290,1.24,1.20,1.0,1.10,30,1


## 3-1-1) 훈련 셋(Train set) 종속변수(Y값) 6월 판매량 기술통계 

In [12]:
m06_SUM = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').sum()[['PD_BUY_CT']]
m06_MEAN = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').mean()[['PD_BUY_CT']]
m06_VAR = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').var()[['PD_BUY_CT']]
m06_MEDIAN = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').median()[['PD_BUY_CT']]
m06_STD = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').std()[['PD_BUY_CT']]
m06_MAX = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').max()[['PD_BUY_CT']]
m06_MIN = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').min()[['PD_BUY_CT']]

In [13]:
m06 = pd.concat([m06_SUM, np.round(m06_MEAN,2), np.round(m06_VAR,2), m06_MEDIAN, np.round(m06_STD,2), m06_MAX, m06_MIN], axis=1)
m06.columns=['m06_SUM', 'm06_MEAN', 'm06_VAR', 'm06_MEDIAN', 'm06_STD', 'm06_MAX', 'm06_MIN']
m06

,m06_SUM,m06_MEAN,m06_VAR,m06_MEDIAN,m06_STD,m06_MAX,m06_MIN
CLAC3_NM,,,,,,,
2단우산,46,1.07,0.07,1.0,0.26,2,1
3단우산,267,1.04,0.05,1.0,0.21,3,1
BB/파운데이션/컴팩트류,12012,1.09,0.22,1.0,0.47,16,1
DIY가구,550,18.97,2046.68,1.0,45.24,173,1
DSLR,115,1.00,0.00,1.0,0.00,1,1
LCD,9,1.00,0.00,1.0,0.00,1,1
LED,344,1.02,0.03,1.0,0.16,3,1
OLED,4,1.00,0.00,1.0,0.00,1,1
PC부품,716,1.21,0.38,1.0,0.61,7,1


In [14]:
m06.to_csv('June.csv', index=True, encoding='CP949')

## 3-2) 7월 ~ 8월 상품군별(상품소분류 기준) 판매량 기술통계 테이블

In [15]:
m0708_SUM = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').sum()[['PD_BUY_CT']]
m0708_MEAN = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').mean()[['PD_BUY_CT']]
m0708_VAR = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').var()[['PD_BUY_CT']]
m0708_MEDIAN = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').median()[['PD_BUY_CT']]
m0708_STD = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').std()[['PD_BUY_CT']]
m0708_MAX = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').max()[['PD_BUY_CT']]
m0708_MIN = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').min()[['PD_BUY_CT']]

In [16]:
m0708 = pd.concat([m0708_SUM, np.round(m0708_MEAN,2), np.round(m0708_VAR,2), m0708_MEDIAN, np.round(m0708_STD,2), m0708_MAX, m0708_MIN], axis=1)
m0708.columns=['m0708_SUM', 'm0708_MEAN', 'm0708_VAR', 'm0708_MEDIAN', 'm0708_STD', 'm0708_MAX', 'm0708_MIN']
m0708

,m0708_SUM,m0708_MEAN,m0708_VAR,m0708_MEDIAN,m0708_STD,m0708_MAX,m0708_MIN
CLAC3_NM,,,,,,,
2단우산,46,1.21,0.33,1.0,0.58,4,1
3단우산,443,1.05,0.05,1.0,0.22,2,1
BB/파운데이션/컴팩트류,22722,1.09,0.17,1.0,0.42,10,1
DIY가구,380,9.74,517.04,1.0,22.74,85,1
DSLR,225,1.01,0.01,1.0,0.12,2,1
LCD,9,1.00,0.00,1.0,0.00,1,1
LED,440,1.02,0.02,1.0,0.15,3,1
OLED,7,1.00,0.00,1.0,0.00,1,1
PC부품,1984,1.27,0.60,1.0,0.77,10,1


## 3-2-1) 테스트 셋(Test set) 종속변수(Y값) 9월 판매량 기술통계 

In [17]:
m09_SUM = mps.query('"20180901" <= SESS_DT  & SESS_DT <= "20180930"').groupby('CLAC3_NM').sum()[['PD_BUY_CT']]
m09_MEAN = mps.query('"20180901" <= SESS_DT  & SESS_DT <= "20180930"').groupby('CLAC3_NM').mean()[['PD_BUY_CT']]
m09_VAR = mps.query('"20180901" <= SESS_DT  & SESS_DT <= "20180930"').groupby('CLAC3_NM').var()[['PD_BUY_CT']]
m09_MEDIAN = mps.query('"20180901" <= SESS_DT  & SESS_DT <= "20180930"').groupby('CLAC3_NM').median()[['PD_BUY_CT']]
m09_STD = mps.query('"20180901" <= SESS_DT  & SESS_DT <= "20180930"').groupby('CLAC3_NM').std()[['PD_BUY_CT']]
m09_MAX = mps.query('"20180901" <= SESS_DT  & SESS_DT <= "20180930"').groupby('CLAC3_NM').max()[['PD_BUY_CT']]
m09_MIN = mps.query('"20180901" <= SESS_DT  & SESS_DT <= "20180930"').groupby('CLAC3_NM').min()[['PD_BUY_CT']]

In [18]:
m09 = pd.concat([m09_SUM, np.round(m09_MEAN,2), np.round(m09_VAR,2), m09_MEDIAN, np.round(m09_STD,2), m09_MAX, m09_MIN], axis=1)
m09.columns=['m09_SUM', 'm09_MEAN', 'm09_VAR', 'm09_MEDIAN', 'm09_STD', 'm09_MAX', 'm09_MIN']
m09

,m09_SUM,m09_MEAN,m09_VAR,m09_MEDIAN,m09_STD,m09_MAX,m09_MIN
CLAC3_NM,,,,,,,
2단우산,4,1.33,0.33,1.0,0.58,2,1
3단우산,121,1.06,0.06,1.0,0.24,2,1
BB/파운데이션/컴팩트류,9993,1.08,0.14,1.0,0.37,11,1
DIY가구,143,14.30,460.46,1.0,21.46,47,1
DIY완구,2,1.00,0.00,1.0,0.00,1,1
DSLR,63,1.00,0.00,1.0,0.00,1,1
LCD,3,1.00,0.00,1.0,0.00,1,1
LED,95,1.01,0.01,1.0,0.10,2,1
OLED,3,1.00,0.00,1.0,0.00,1,1


In [19]:
m09.to_csv('September.csv', index=True, encoding='CP949')

## 3-3) 직전 2개월 (4-5월 / 7-8월) 기술통계 테이블 합치기

In [20]:
two_months = pd.DataFrame(index = mps.groupby('CLAC3_NM').sum().index)
two_months = pd.concat([two_months, m0405, m0708], axis=1)
two_months

C:\Users\gmlwl\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,m0405_SUM,m0405_MEAN,m0405_VAR,m0405_MEDIAN,m0405_STD,m0405_MAX,m0405_MIN,m0708_SUM,m0708_MEAN,m0708_VAR,m0708_MEDIAN,m0708_STD,m0708_MAX,m0708_MIN
2단우산,110.0,1.41,1.47,1.0,1.21,7.0,1.0,46.0,1.21,0.33,1.0,0.58,4.0,1.0
3단우산,806.0,1.20,1.02,1.0,1.01,11.0,1.0,443.0,1.05,0.05,1.0,0.22,2.0,1.0
BB/파운데이션/컴팩트류,24690.0,1.09,0.22,1.0,0.47,20.0,1.0,22722.0,1.09,0.17,1.0,0.42,10.0,1.0
DIY가구,467.0,13.34,554.88,1.0,23.56,92.0,1.0,380.0,9.74,517.04,1.0,22.74,85.0,1.0
DIY완구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DSLR,227.0,1.00,0.00,1.0,0.00,1.0,1.0,225.0,1.01,0.01,1.0,0.12,2.0,1.0
LCD,2.0,1.00,0.00,1.0,0.00,1.0,1.0,9.0,1.00,0.00,1.0,0.00,1.0,1.0
LED,536.0,1.06,0.39,1.0,0.63,9.0,1.0,440.0,1.02,0.02,1.0,0.15,3.0,1.0
OLED,2.0,1.00,0.00,1.0,0.00,1.0,1.0,7.0,1.00,0.00,1.0,0.00,1.0,1.0
PC부품,1290.0,1.24,1.20,1.0,1.10,30.0,1.0,1984.0,1.27,0.60,1.0,0.77,10.0,1.0


## 3-4) 직전 2개월간 한번도 구매되지 않은 제품 출력

In [21]:
# 4-5월에 구매가 한번도 되지 않은 상품소분류명 출력
two_months[np.isnan(two_months['m0405_SUM'])].index

Index(['DIY완구', '감', '과실주병', '기저귀크림/파우더', '무화과', '반찬세트', '방한모', '복숭아', '볶음반찬',
       '살구', '석류', '세탁기소모품', '수도용품', '육가공품선물세트', '차량용충전기'],
      dtype='object')

In [22]:
# 7-8월에 구매가 한번도 되지 않은 상품소분류명 출력
two_months[np.isnan(two_months['m0708_SUM'])].index

Index(['DIY완구', '감', '남성등산스웨트셔츠/후드/집업', '냉동디저트', '미용거울', '반찬세트', '석류', '싱크대',
       '아이젠', '인라인/스케이트보드/킥보드기타액세서리', '차량용충전기'],
      dtype='object')

## 3-5) 직전 2개월 테이블 csv저장

In [23]:
two_months.to_csv('The_last_two_months.csv', index=True, encoding='CP949')

<hr>

# 4. 직전 3개월 데이터 기술통계 테이블 생성 (2번 테이블)
## 4-1) 4월 / 5월 / 6월 상품군별(상품소분류 기준) 판매량 기술통계 테이블

In [24]:
m456_SUM = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').sum()[['PD_BUY_CT']]
m456_MEAN = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').mean()[['PD_BUY_CT']]
m456_VAR = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').var()[['PD_BUY_CT']]
m456_MEDIAN = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').median()[['PD_BUY_CT']]
m456_STD = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').std()[['PD_BUY_CT']]
m456_MAX = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').max()[['PD_BUY_CT']]
m456_MIN = mps.query('"20180401" <= SESS_DT  & SESS_DT <= "20180630"').groupby('CLAC3_NM').min()[['PD_BUY_CT']]

In [25]:
m456 = pd.concat([m456_SUM, np.round(m456_MEAN,2), np.round(m456_VAR,2), m456_MEDIAN, np.round(m456_STD,2), m456_MAX, m456_MIN], axis=1)
m456.columns=['m456_SUM', 'm456_MEAN', 'm456_VAR', 'm456_MEDIAN', 'm456_STD', 'm456_MAX', 'm456_MIN']
m456

,m456_SUM,m456_MEAN,m456_VAR,m456_MEDIAN,m456_STD,m456_MAX,m456_MIN
CLAC3_NM,,,,,,,
2단우산,156,1.29,0.99,1.0,1.00,7,1
3단우산,1073,1.16,0.76,1.0,0.87,11,1
BB/파운데이션/컴팩트류,36702,1.09,0.22,1.0,0.47,20,1
DIY가구,1017,15.89,1217.05,1.0,34.89,173,1
DSLR,342,1.00,0.00,1.0,0.00,1,1
LCD,11,1.00,0.00,1.0,0.00,1,1
LED,880,1.04,0.25,1.0,0.50,9,1
OLED,6,1.00,0.00,1.0,0.00,1,1
PC부품,2006,1.23,0.90,1.0,0.95,30,1


## 4-1-1) 훈련 셋(Train set) 종속변수(Y값) 7월 판매량 기술통계

In [26]:
m07_SUM = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').sum()[['PD_BUY_CT']]
m07_MEAN = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').mean()[['PD_BUY_CT']]
m07_VAR = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').var()[['PD_BUY_CT']]
m07_MEDIAN = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').median()[['PD_BUY_CT']]
m07_STD = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').std()[['PD_BUY_CT']]
m07_MAX = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').max()[['PD_BUY_CT']]
m07_MIN = mps.query('"20180701" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').min()[['PD_BUY_CT']]

In [27]:
m07 = pd.concat([m07_SUM, np.round(m07_MEAN,2), np.round(m07_VAR,2), m07_MEDIAN, np.round(m07_STD,2), m07_MAX, m07_MIN], axis=1)
m07.columns=['m07_SUM', 'm07_MEAN', 'm07_VAR', 'm07_MEDIAN', 'm07_STD', 'm07_MAX', 'm07_MIN']
m07

,m07_SUM,m07_MEAN,m07_VAR,m07_MEDIAN,m07_STD,m07_MAX,m07_MIN
CLAC3_NM,,,,,,,
2단우산,30,1.25,0.46,1.0,0.68,4,1
3단우산,247,1.06,0.06,1.0,0.24,2,1
BB/파운데이션/컴팩트류,11251,1.10,0.18,1.0,0.43,10,1
DIY가구,214,10.19,519.96,1.0,22.80,78,1
DSLR,140,1.01,0.01,1.0,0.12,2,1
LED,288,1.02,0.02,1.0,0.14,2,1
OLED,6,1.00,0.00,1.0,0.00,1,1
PC부품,804,1.23,0.45,1.0,0.67,10,1
UHD,85,1.01,0.01,1.0,0.11,2,1


In [28]:
m07.to_csv('July.csv', index=True, encoding='CP949')

## 4-2) 5월/ 6월/ 7월 상품군별(상품소분류 기준) 판매량 기술통계 테이블

In [29]:
m567_SUM = mps.query('"20180501" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').sum()[['PD_BUY_CT']]
m567_MEAN = mps.query('"20180501" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').mean()[['PD_BUY_CT']]
m567_VAR = mps.query('"20180501" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').var()[['PD_BUY_CT']]
m567_MEDIAN = mps.query('"20180501" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').median()[['PD_BUY_CT']]
m567_STD = mps.query('"20180501" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').std()[['PD_BUY_CT']]
m567_MAX = mps.query('"20180501" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').max()[['PD_BUY_CT']]
m567_MIN = mps.query('"20180501" <= SESS_DT  & SESS_DT <= "20180731"').groupby('CLAC3_NM').min()[['PD_BUY_CT']]

In [30]:
m567 = pd.concat([m567_SUM, np.round(m567_MEAN,2), np.round(m567_VAR,2), m567_MEDIAN, np.round(m567_STD,2), m567_MAX, m567_MIN], axis=1)
m567.columns=['m567_SUM', 'm567_MEAN', 'm567_VAR', 'm567_MEDIAN', 'm567_STD', 'm567_MAX', 'm567_MIN']
m567

,m567_SUM,m567_MEAN,m567_VAR,m567_MEDIAN,m567_STD,m567_MAX,m567_MIN
CLAC3_NM,,,,,,,
2단우산,120,1.22,0.57,1.0,0.75,6,1
3단우산,878,1.07,0.10,1.0,0.31,4,1
BB/파운데이션/컴팩트류,36567,1.09,0.19,1.0,0.43,16,1
DIY가구,1170,16.71,1220.55,1.0,34.94,173,1
DSLR,374,1.01,0.01,1.0,0.07,2,1
LCD,10,1.00,0.00,1.0,0.00,1,1
LED,893,1.05,0.24,1.0,0.49,9,1
OLED,11,1.00,0.00,1.0,0.00,1,1
PC부품,2178,1.22,0.43,1.0,0.65,10,1


## 4-2-1) 훈련 셋(Train set) 종속변수(Y값) 8월 판매량 기술통계 

In [31]:
m08_SUM = mps.query('"20180801" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').sum()[['PD_BUY_CT']]
m08_MEAN = mps.query('"20180801" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').mean()[['PD_BUY_CT']]
m08_VAR = mps.query('"20180801" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').var()[['PD_BUY_CT']]
m08_MEDIAN = mps.query('"20180801" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').median()[['PD_BUY_CT']]
m08_STD = mps.query('"20180801" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').std()[['PD_BUY_CT']]
m08_MAX = mps.query('"20180801" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').max()[['PD_BUY_CT']]
m08_MIN = mps.query('"20180801" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').min()[['PD_BUY_CT']]

In [32]:
m08 = pd.concat([m08_SUM, np.round(m08_MEAN,2), np.round(m08_VAR,2), m08_MEDIAN, np.round(m08_STD,2), m08_MAX, m08_MIN], axis=1)
m08.columns=['m08_SUM', 'm08_MEAN', 'm08_VAR', 'm08_MEDIAN', 'm08_STD', 'm08_MAX', 'm08_MIN']
m08

,m08_SUM,m08_MEAN,m08_VAR,m08_MEDIAN,m08_STD,m08_MAX,m08_MIN
CLAC3_NM,,,,,,,
2단우산,16,1.14,0.13,1.0,0.36,2,1
3단우산,196,1.04,0.04,1.0,0.19,2,1
BB/파운데이션/컴팩트류,11471,1.09,0.16,1.0,0.40,10,1
DIY가구,166,9.22,543.48,1.0,23.31,85,1
DSLR,85,1.01,0.01,1.0,0.11,2,1
LCD,9,1.00,0.00,1.0,0.00,1,1
LED,152,1.01,0.03,1.0,0.16,3,1
OLED,1,1.00,NaN,1.0,NaN,1,1
PC부품,1180,1.29,0.70,1.0,0.84,10,1


In [33]:
m08.to_csv('August.csv', index=True, encoding='CP949')

## 4-3) 6월/ 7월/ 8월 상품군별(상품소분류 기준) 판매량 기술통계 테이블

In [34]:
m678_SUM = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').sum()[['PD_BUY_CT']]
m678_MEAN = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').mean()[['PD_BUY_CT']]
m678_VAR = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').var()[['PD_BUY_CT']]
m678_MEDIAN = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').median()[['PD_BUY_CT']]
m678_STD = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').std()[['PD_BUY_CT']]
m678_MAX = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').max()[['PD_BUY_CT']]
m678_MIN = mps.query('"20180601" <= SESS_DT  & SESS_DT <= "20180831"').groupby('CLAC3_NM').min()[['PD_BUY_CT']]

In [35]:
m678 = pd.concat([m678_SUM, np.round(m678_MEAN,2), np.round(m678_VAR,2), m678_MEDIAN, np.round(m678_STD,2), m678_MAX, m678_MIN], axis=1)
m678.columns=['m678_SUM', 'm678_MEAN', 'm678_VAR', 'm678_MEDIAN', 'm678_STD', 'm678_MAX', 'm678_MIN']
m678

,m678_SUM,m678_MEAN,m678_VAR,m678_MEDIAN,m678_STD,m678_MAX,m678_MIN
CLAC3_NM,,,,,,,
2단우산,92,1.14,0.19,1.0,0.44,4,1
3단우산,710,1.05,0.05,1.0,0.22,3,1
BB/파운데이션/컴팩트류,34734,1.09,0.19,1.0,0.43,16,1
DIY가구,930,13.68,1169.68,1.0,34.20,173,1
DSLR,340,1.01,0.01,1.0,0.09,2,1
LCD,18,1.00,0.00,1.0,0.00,1,1
LED,784,1.02,0.02,1.0,0.16,3,1
OLED,11,1.00,0.00,1.0,0.00,1,1
PC부품,2700,1.25,0.54,1.0,0.73,10,1


In [36]:
three_months = pd.DataFrame(index = mps.groupby('CLAC3_NM').sum().index)
three_months = pd.concat([three_months, m456, m567, m678], axis=1)
three_months

C:\Users\gmlwl\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,m456_SUM,m456_MEAN,m456_VAR,m456_MEDIAN,m456_STD,m456_MAX,m456_MIN,m567_SUM,m567_MEAN,m567_VAR,...,m567_STD,m567_MAX,m567_MIN,m678_SUM,m678_MEAN,m678_VAR,m678_MEDIAN,m678_STD,m678_MAX,m678_MIN
2단우산,156.0,1.29,0.99,1.0,1.00,7.0,1.0,120.0,1.22,0.57,...,0.75,6.0,1.0,92.0,1.14,0.19,1.0,0.44,4.0,1.0
3단우산,1073.0,1.16,0.76,1.0,0.87,11.0,1.0,878.0,1.07,0.10,...,0.31,4.0,1.0,710.0,1.05,0.05,1.0,0.22,3.0,1.0
BB/파운데이션/컴팩트류,36702.0,1.09,0.22,1.0,0.47,20.0,1.0,36567.0,1.09,0.19,...,0.43,16.0,1.0,34734.0,1.09,0.19,1.0,0.43,16.0,1.0
DIY가구,1017.0,15.89,1217.05,1.0,34.89,173.0,1.0,1170.0,16.71,1220.55,...,34.94,173.0,1.0,930.0,13.68,1169.68,1.0,34.20,173.0,1.0
DIY완구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DSLR,342.0,1.00,0.00,1.0,0.00,1.0,1.0,374.0,1.01,0.01,...,0.07,2.0,1.0,340.0,1.01,0.01,1.0,0.09,2.0,1.0
LCD,11.0,1.00,0.00,1.0,0.00,1.0,1.0,10.0,1.00,0.00,...,0.00,1.0,1.0,18.0,1.00,0.00,1.0,0.00,1.0,1.0
LED,880.0,1.04,0.25,1.0,0.50,9.0,1.0,893.0,1.05,0.24,...,0.49,9.0,1.0,784.0,1.02,0.02,1.0,0.16,3.0,1.0
OLED,6.0,1.00,0.00,1.0,0.00,1.0,1.0,11.0,1.00,0.00,...,0.00,1.0,1.0,11.0,1.00,0.00,1.0,0.00,1.0,1.0
PC부품,2006.0,1.23,0.90,1.0,0.95,30.0,1.0,2178.0,1.22,0.43,...,0.65,10.0,1.0,2700.0,1.25,0.54,1.0,0.73,10.0,1.0


## 4-4) 직전 3개월간 한번도 구매되지 않은 제품 출력

In [37]:
# 4-6월에 구매가 한번도 되지 않은 상품소분류명 출력
three_months[np.isnan(three_months['m456_SUM'])].index

Index(['DIY완구', '과실주병', '무화과', '반찬세트', '방한모', '석류', '수도용품', '육가공품선물세트'], dtype='object')

In [38]:
# 5-7월에 구매가 한번도 되지 않은 상품소분류명 출력
three_months[np.isnan(three_months['m567_SUM'])].index

Index(['DIY완구', '과실주병', '무화과', '반찬세트', '발', '방한모', '석류', '수도용품', '아이젠',
       '육가공품선물세트'],
      dtype='object')

In [39]:
# 6-8월에 구매가 한번도 되지 않은 상품소분류명 출력
three_months[np.isnan(three_months['m678_SUM'])].index

Index(['DIY완구', '냉동디저트', '미용거울', '반찬세트', '석류', '싱크대', '아이젠'], dtype='object')

## 4-5) 직전 3개월 테이블 csv 저장

In [40]:
three_months.to_csv('The_last_three_months.csv', index=True, encoding='CP949')